In [0]:
#Customers who bought All Products

In [0]:
from pyspark.sql.functions import countDistinct,count,col

customer_data=[(1,5),(2,6),(3,5),(3,6),(1,6)]

product_data=[(5,),(6,)]

#create dataframes for Customer and Product tables
customer_df = spark.createDataFrame(customer_data,["customer_id","product_key"])

product_df = spark.createDataFrame(product_data,["productkey"])

customer_df.display()
product_df.display()

customer_id,product_key
1,5
2,6
3,5
3,6
1,6


productkey
5
6


In [0]:
customers_product_count = customer_df.groupBy("customer_id").agg(count("product_key").alias("product_count")) 

total_product_count = product_df.count()

#customers who bought all products
result_df = customers_product_count.filter(
    customers_product_count.product_count == total_product_count
)

result_df.display()

customer_id,product_count
1,2
3,2


In [0]:
#or

# Get the total distinct product count
total_product_count = product_df.select("productkey").distinct().count()

# Combine grouping, aggregation, and filtering in one statement
result_df = customer_df \
    .groupBy("customer_id") \
    .agg(count("product_key").alias("product_count")) \
    .filter(col("product_count") == total_product_count)


result_df.display()

customer_id,product_count
1,2
3,2


###using Spark SQL

In [0]:
customer_df.createOrReplaceTempView("customers")
product_df.createOrReplaceTempView("products")

In [0]:
result_df =spark.sql(
 """
    select customer_id, count(distinct product_key) as product_count
    from customers
    group by customer_id
    having count(distinct product_key) = (select count(distinct productkey)from products)
"""
)
result_df.show()


+-----------+-------------+
|customer_id|product_count|
+-----------+-------------+
|          1|            2|
|          3|            2|
+-----------+-------------+



In [0]:
%sql

--or

with cte1 as ( 
  select customer_id, count(distinct product_key) as product_count
  from customers
  group by customer_id
)
,cte2 as (
  select count(distinct productkey) as product_count from products
)

select customer_id
from cte1 join cte2 on cte1.product_count=cte2.product_count

customer_id
1
3


In [0]:
%sql

--or 

with cte1 as (
  select customer_id, count(distinct product_key) as product_count
  from customers
  group by customer_id
)

select customer_id
from cte1
where product_count = (select count(distinct productkey) as product_count from products)


customer_id
1
3
